In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager
from tmqrfeed.contracts import FutureContract, OptionContract
from tmqrfeed.quotes.quote_contfut import QuoteContFut
from tmqr.errors import *

In [ ]:
%load_ext line_profiler
import line_profiler
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler import Options

directive_defaults = Options.get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [ ]:
%memit

In [ ]:
dm = DataManager()

# DataManager chain getting

In [ ]:
dt = datetime(2011, 12, 21)
dm.chains_futures_get('US.CL', dt, offset=0)

In [ ]:
%memit

In [ ]:
dm.series_primary_set(QuoteContFut, 'US.CL', datamanager=dm, timeframe='D' )

In [ ]:
%memit

In [ ]:
quotes = dm._primary_quotes

dates = quotes.ix['2011-12': '2012-01-15'].index

In [ ]:
def request_quotes():
    for dt in dates:
        try: 
            fut, opt_chain = dm.chains_options_get('US.CL', dt, opt_offset=0)
            #opt = opt_chain.find(dt, 0, 'C', how='offset')
            opt = opt_chain.find(dt, 0.2, 'C', how='delta')            
            #opt = opt_chain.find(dt, 0.5, 'C', how='delta')
            #opt = opt_chain.find(dt, 0.8, 'C', how='delta')
        except NotFoundError as exc:
            print(f"{type(exc)}: {str(exc)}")

In [ ]:
%lprun -f dm.price_get request_quotes()

In [ ]:
%lprun -m tmqrfeed.datafeed request_quotes()

In [ ]:
%lprun -m tmqrfeed.chains request_quotes()

In [ ]:
%lprun -f request_quotes request_quotes()

In [ ]:
%timeit request_quotes()

In [ ]:
%memit request_quotes()

### Timings (dm.chains_options_get) without caching - per hit 134ms

### Timings (dm.chains_options_get) fut_chain caching - per hit 27ms (5x speed up)

### Timings (dm.chains_options_get) opt_chain caching - per hit 3ms (45x speed up)

### Timings (dm.chains_options_get) fut_chain_get_contract refactored - per hit 78nanosec (1700x speed up)

In [ ]:
### Timings (dm.chains_options_find) opt_chain.find - datafeed cache added - per hit 16ms was 79 (5x speed up)

# Overall speed up 

In [ ]:
slow = 3850 # Initial version without caching and with bottlenecks
last = 226
print(f"Speedup: {slow/last:0.2f} times")